# Cellular Automata

Finite state machines across space!

Rule 90: Discrete form of the logistic map: $x_{t+1} = r x_t (1 - x_t)$.

$$x_{i,t+1} = r (x_{i-1,t} + x_{i+1,t}) (2 - (x_{i-1,t} + x_{i+1,t})$$

If $r = 1$, simplifes to a set of rules:
 - If $x_{i-1,t} + x_{i+1,t} = 0$, then $x_{i,t+1} = 0$
 - If $x_{i-1,t} + x_{i+1,t} = 2$, then $x_{i,t+1} = 0$
 - If $x_{i-1,t} + x_{i+1,t} = 1$, then $x_{i,t+1} = 1$

In [ ]:
xx = matrix(0, 99, 100)
xx[50, 1] = 1

In [ ]:
image(xx)

In [ ]:
for (tt in 2:100) {
    for (ii in 2:98) {
        xx[ii, tt] = (xx[ii-1, tt-1] + xx[ii+1, tt-1])*(2 - (xx[ii-1, tt-1] + xx[ii+1, tt-1]))
    }
}

In [ ]:
image(xx)

# Sandpile model

Version 1: Add random grains

In [ ]:
cells = matrix(0, 40, 40)
for (ii in 1:1000) {
    rc = sample(40*40, 1)
    cells[rc] = cells[rc] + 1
}

In [ ]:
image(cells, zlim=c(0, 4))

In [ ]:
table(cells)

In [ ]:
which(cells >= 4)

In [ ]:
coords = arrayInd(which(cells >= 4), dim(cells))

In [ ]:
coords

In [ ]:
cells[coords[,1], coords[,2]] = cells[coords[,1], coords[,2]] - 4
notleft = coords[,1] > 1
notright = coords[,1] < 40
notbottom = coords[,2] > 1
nottop = coords[,2] < 40
cells[coords[notleft,1] - 1, coords[notleft,2]] = cells[coords[notleft,1] - 1, coords[notleft,2]] + 1
cells[coords[notright,1] + 1, coords[notright,2]] = cells[coords[notright,1] + 1, coords[notright,2]] + 1
cells[coords[notbottom,1], coords[notbottom,2] - 1] = cells[coords[notbottom,1], coords[notbottom,2] - 1] + 1
cells[coords[nottop,1], coords[nottop,2] + 1] = cells[coords[nottop,1], coords[nottop,2] + 1] + 1

In [ ]:
image(cells, zlim=c(0, 4))

Version 2: Include this in the loop

In [ ]:
size = 40
cells = matrix(sample(4, size*size, replace=T) - 1, size, size)
numtoppled = c()
for (ii in 1:10000) {
    rc = sample(size*size, 1)
    cells[rc] = cells[rc] + 1
    thistoppled = 0
    while (any(cells >= 4)) {
        coords = arrayInd(which(cells >= 4), dim(cells))
        cells[coords[,1], coords[,2]] = cells[coords[,1], coords[,2]] - 4
        notleft = coords[,1] > 1
        notright = coords[,1] < 40
        notbottom = coords[,2] > 1
        nottop = coords[,2] < 40
        cells[coords[notleft,1] - 1, coords[notleft,2]] = cells[coords[notleft,1] - 1, coords[notleft,2]] + 1
        cells[coords[notright,1] + 1, coords[notright,2]] = cells[coords[notright,1] + 1, coords[notright,2]] + 1
        cells[coords[notbottom,1], coords[notbottom,2] - 1] = cells[coords[notbottom,1], coords[notbottom,2] - 1] + 1
        cells[coords[nottop,1], coords[nottop,2] + 1] = cells[coords[nottop,1], coords[nottop,2] + 1] + 1
        thistoppled = thistoppled + sum(cells >= 4)
    }
    numtoppled = c(numtoppled, thistoppled)
}

In [ ]:
sum(numtoppled)

In [ ]:
plot(numtoppled[numtoppled > 0])

In [ ]:
plot(numtoppled[numtoppled > 0 & numtoppled < 100])

In [ ]:
library(ggplot2)
ggplot(data.frame(nn=numtoppled[numtoppled > 0]), aes(nn)) +
  geom_histogram()

In [ ]:
library(ggplot2)
ggplot(data.frame(nn=numtoppled), aes(nn)) +
  geom_histogram() + scale_x_log10() + scale_y_log10()

In [ ]:
library(ggplot2)
ggplot(data.frame(nn=numtoppled[numtoppled < 100]), aes(nn)) +
  geom_histogram() + scale_x_log10() + scale_y_log10()

# Finding power law exponents

In [ ]:
install.packages("VGAM")
library(VGAM)

In [ ]:
ggplot(data.frame(x=rpareto(1e6, shape=1)), aes(x)) + geom_histogram() + scale_x_log10() + scale_y_log10()

In [ ]:
values = rpareto(1e6, shape=1)

In [ ]:
bins = seq(log(1), log(10000), length.out=100)

In [ ]:
bins

In [ ]:
count = c()
for (ii in 2:length(bins)) {
    count = c(count, sum(log(valeus) >= bins[ii-1] & log(valeus) < bins[ii]))
}

In [ ]:
count

In [ ]:
count = table(cut(log(values), bins))

In [ ]:
count

In [ ]:
logct = log(count)
logct[!is.finite(logct)] = NA
midbn = (bins[-1] + bins[-length(bins)]) / 2

In [ ]:
ggplot(data.frame(logct, midbn), aes(midbn, logct)) + geom_point() + geom_smooth(method='lm')

In [ ]:
summary(lm(logct ~ midbn))

## Fit by maximum likelihood

Power law distribution is:
$$p(x | \beta) = C x^{-\beta}$$

We have to know $C$, so calculate:
$$\int_{x_{min}}^\infty C x^{-\beta} dx = 1$$

Find $C = \frac{-\beta + 1}{x_{min}^{-\beta + 1}}$

Simplify the power law to:
$$p(x | \beta) = \frac{-\beta + 1}{x_{min}} \left(\frac{x}{x_{min}}\right)^{-\beta}$$

Solve for maximum likelihood of
$$\prod_i p(x_i | \beta)$$

Get $$\hat\beta = n \left(\sum_i \log{\left(\frac{x_i}{x_{min}}\right)}\right)^{-1}$$

In [ ]:
betahat = function(sizes, xmin) {
    length(sizes) / sum(log(sizes / xmin))
}

In [ ]:
betahat(valeus, 1)

In [ ]:
betahat(valeus[valeus > 10], 10)

# Looking at SOC data

In [ ]:
df = read.csv("soc-avalanche.csv")

In [ ]:
nrow(df)

In [ ]:
ggplot(df, aes(size)) + geom_histogram()

In [ ]:
ggplot(df, aes(size)) + geom_histogram() + scale_x_log10() + scale_y_log10()

In [ ]:
bins = seq(log(1), log(10000), length.out=100)

In [ ]:
count = c()
for (ii in 2:length(bins)) {
    count = c(count, sum(log(df$size) >= bins[ii-1] & log(df$size) < bins[ii]))
}

In [ ]:
plot(bins[-1], count, log='y')

In [ ]:
logct = log(count)
logct[!is.finite(logct)] = NA
midbn = (bins[-1] + bins[-length(bins)]) / 2

In [ ]:
summary(lm(logct ~ midbn))

$$\log{Count} = m \log{Value} + b$$

In [ ]:
ggplot(data.frame(logct, midbn), aes(midbn, logct)) + geom_point() + geom_smooth(method='lm')

In [ ]:
betahat(df$size, 1)

In [ ]:
betahat(df$size[df$size < exp(2.5)], 1)

In [ ]:
betahat(df$size[df$size > exp(2.5) & df$size < exp(8)], exp(2.5))